# Who votes more often with the right?

Who votes with whom and which political groups form majorities in plenary votes is often of particular interest – not only to researchers, but also in public discourse.

In his July 2025 "Summer Interview" with German public broadcaster ARD, Friedrich Merz claimed that the Group of the Socialists and Democrats (S&D) had voted with right-wing groups more often than the Group of the European People’s Party (EPP), around 13:30 in the [interview](https://www.ardmediathek.de/video/bericht-aus-berlin/ard-sommerinterview-friedrich-merz/ard/Y3JpZDovL2Rhc2Vyc3RlLmRlL2JlcmljaHQgYXVzIGJlcmxpbi8yMDI1LTA3LTEzXzE4LTAwLU1FU1o).

In this tutorial, we will demonstrate how to fact-check this claim.

To follow along with the tutorial, you should already be familiar with data analysis in Python using the `pandas` package. You don’t need prior knowledge about the European Parliament.

## Definitions

The trickiest part in conducting an analysis like this is defining what it means for a group "to vote with another group".
We propose to define the group position to be the majority position within a group. For example, the group position of a group where 10 MEPs voted in favor, 8 voted against, and 3 abstained is "in favor".
For any given vote, two groups are considered to have voted together if they share the same group position.
Note that this does not necessarily imply that the vote is successful.
We decided on this approach, as Merz's statement did not mention majorities or a limitation to only successful vote, but instead referred only to groups voting with the right.

Furthermore, Merz's statement referred to "groups on the right". We consider these to be the groups of the European Conservatives and Reformists (ECR), Patriots for Europe (PfE), and Europe of Sovereign Nations (ESN), as these are generally considered right of the EPP.
We will focus on the 10th parliamentary term, the term during which Merz made his claim. 

In the following, we will calculate how often S&D and EPP shared the same group position with each of these right-wing groups.

## Roll-call votes

It is important to keep in mind that only some of the votes in the European Parliament are roll-call votes.
The voting position of individual MEPs is only recorded and published in case of a roll-call vote.
Roll-call votes are quite common in the European Parliament, but not ubiquitous. Naturally, for votes that are not cast by roll-call it is impossible for us (or anyone else) to know exactly which MEPs and groups voted similarly.

## Overview

We will follow these steps:

1. Find all votes which took place during the 10th term of Parliament and exclude some special cases like votes on the agenda.
2. For each vote, construct a table in which we sum up the votes per group per position. This will result in a table where we have one row per vote-group-position combination, and the number of MEPs who voted that way.
3. Using this table, we can find which position per vote was most common among MEPs of each group - this is the group position.
4. We will then construct a table with only a single row per vote and a column per group containing the group position.
5. Finally, we can compare the columns, allowing us to count how often two given groups shared the same group position.

## Step 1: Filtering for relevant votes

In [ ]:
import pandas as pd

# The votes table contains one row per roll-call vote
votes_df = pd.read_csv('data/votes.csv')
votes_df.head()

In [ ]:
votes_df['timestamp'] = pd.to_datetime(votes_df['timestamp'], format='ISO8601')

# Filter votes keeping only votes during the 10th term and before Merz's statement
mask_time = (votes_df["timestamp"] >= "2024-07-16") & (votes_df["timestamp"] <= "2025-07-13")
votes_subset = votes_df[mask_time]

# Filter votes to exclude special cases like votes on the agenda
exclude_pattern = "Ordre du jour|Demande du group|Demande des groupes|Request from the"
mask_title = votes_subset['display_title'].str.contains(exclude_pattern)
votes_subset = votes_subset[~mask_title]
votes_subset.head()

## Step 2: Aggregating by group, vote, and position

The `votes` table does not contain information about the individual votes of MEPs. This information can be found in the `member_votes` table. This table contains one row per vote and MEP, with columns for their vote position, group membership at the time of the vote, and nationality.

In [ ]:
member_votes_df = pd.read_csv('data/member_votes.csv')
member_votes_df.head()

We use a filtering join with the `votes_subset` dataframe to only keep member votes related to the subset of votes we selected previously:

In [ ]:
member_votes_subset = member_votes_df[member_votes_df["vote_id"].isin(votes_subset["id"])]
member_votes_subset.head()

Before aggregating by group and position, we can exclude a lot of rows from the `member_votes` table. As we are only interested in the voting behavior of the EPP and S&D groups as well as the three right-wing groups, we can exclude all rows related to MEPs of other groups.

In [ ]:
member_votes_subset = member_votes_subset[member_votes_subset["group_code"].isin(["EPP", "SD", "ECR", "PFE", "ESN"])]
member_votes_subset.head()

Next, we construct a table with one row per vote-group-position combination and the number of MEPs corresponding to each of these combinations.
Technically, this comes down to grouping by the `vote_id`, `group_code`, and `position` columns, and then adding the size of the group as a new column.

In [ ]:
group_counts = member_votes_subset \
    .groupby(["vote_id", "group_code", "position"]) \
    .size() \
    .reset_index(name="count")

group_counts

## Step 3: Identifying group positions

Next, we want to identify the majority position within each group.
The code below achieves this by first sorting by number of MEPs in ascending order, so that the vote-group-position combination with the highest number of MEPs comes first. It then removes duplicates based on vote-group combinations, leaving only the first entry per vote-group combination – which represents the group position.

In [ ]:
group_position = group_counts \
    .sort_values("count", ascending=False) \
    .drop_duplicates(subset=["vote_id","group_code"])

group_position

## Step 4: Pivoting

As a last step, we will pivot this table, resulting in a single row per vote, and one column per group containing the group position.

In [ ]:
group_position_comparison = (
    group_position
    .pivot(index="vote_id", columns="group_code", values="position")
    .reset_index()
)
group_position_comparison

## Step 5: Comparing group positions

We can now easily compare the group positions of the EPP and S&D groups with each of the three right-wing group, starting with the ESN group.
We will add two separate boolean columns indicating whether the ESN’s group position was the same as the S&D and EPP group position, respectively.

In [ ]:
group_position_comparison["EPP_equals_ESN"] = group_position_comparison["EPP"] == group_position_comparison["ESN"]
group_position_comparison["SD_equals_ESN"]  = group_position_comparison["SD"]  == group_position_comparison["ESN"]

To find out whether the EPP or S&D group voted more often with the ESN group, we can simply count the number of `True` values in each column.

In [ ]:
sum(group_position_comparison["EPP_equals_ESN"])

In [ ]:
sum(group_position_comparison["SD_equals_ESN"])

In the 10th term of the European Parliament, the EPP voted with the ESN group more often than the S&D group.

Next, we compare the EPP and S&D groups with the PfE group:

In [ ]:
group_position_comparison["EPP_equals_PFE"] = group_position_comparison["EPP"] == group_position_comparison["PFE"]
group_position_comparison["SD_equals_PFE"]  = group_position_comparison["SD"]  == group_position_comparison["PFE"]

In [ ]:
sum(group_position_comparison["EPP_equals_PFE"])

In [ ]:
sum(group_position_comparison["SD_equals_PFE"])

Again, the EPP voted with the PfE group more often than the S&D group.

Finally, we compare both groups with the ECR group:

In [ ]:
group_position_comparison["EPP_equals_ECR"] = group_position_comparison["EPP"] == group_position_comparison["ECR"]
group_position_comparison["SD_equals_ECR"]  = group_position_comparison["SD"]  == group_position_comparison["ECR"]

In [ ]:
sum(group_position_comparison["EPP_equals_ECR"])

In [ ]:
sum(group_position_comparison["SD_equals_ECR"])

Again, the EPP group voted more often with the ECR group than the S&D group.

## Conclusion

Considering only roll-call votes of the 10th term of the European Parliament, the EPP voted together with right-wing groups more often than the S&D group. To the extent Merz's claim is verifiable, it is false.